In [34]:
from pathlib import Path 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np 
import json 
import plot_utils

plot_utils.set_style()
plt.ioff()

In [35]:
result_dir = Path('../results')
data_dir = Path('../data')

method_order = {'Ours': 0, 'pTS': 1, 'qEI': 2, 'Greedy': 3, 'UCB': 4}


# Maximizing homo-lumo gap 

### Loading, calculating

In [36]:
storage = []
for file in result_dir.glob('qm9*.json'): 
    with open(file, 'r') as f: 
        storage.extend(json.load(f))

Identify true top k and add to storage fraction of true top k information.

In [37]:
all_data = pd.read_csv(data_dir / 'qm9.csv')
all_smiles = list(all_data['smiles'])
test_data = {smi: score for smi, score in zip(all_smiles, all_data['gap'])}

k = [0.0001, 0.001, 0.005, 0.01]
kN = [int(np.ceil(kk*len(test_data))) for kk in k]
top_smis = {kk: set(sorted(list(test_data), key = lambda smi: -1*test_data[smi])[:kk]) for kk in kN}

for entry in storage: 
    for kkN, kk in zip(kN, k): 
        entry[f'Fraction top {100*kk:0.2f}%'] = len(top_smis[kkN].intersection(set(entry["All acquired points"].keys())))/kkN
    entry.pop("All acquired points")
    entry.pop("New acquired points")

data = pd.DataFrame(storage).sort_values(by='Method', key = lambda x: x.map(method_order))

### Plotting

In [38]:
fig_dir = Path('qm9')
fig_dir.mkdir(exist_ok=True, parents=True)

### Average of top k acquired 

In [40]:
for kk in [1, 10, 50, 100]:
    fig, ax = plt.subplots(1,1)
    sns.lineplot(
        data=data, x='Iteration', y=f'Top {kk} ave', 
        hue='Method', style='Method', palette=plot_utils.method_colors,
        ax=ax, errorbar=('se',1))
    ax.set_ylabel(f'Average of top {kk} acquired')
    plot_utils.set_size(w=1.2, h=1, ax=ax)
    fig.savefig(fig_dir / f'top_{kk}_average.pdf', bbox_inches='tight', transparent=True)



### Fraction of true top k 

In [41]:
for kk, kkN in zip(k, kN):
    fig, ax = plt.subplots(1,1)
    sns.lineplot(
        data=data, x='Iteration', y=f'Fraction top {100*kk:0.2f}%', 
        hue='Method', style='Method', palette=plot_utils.method_colors,
        ax=ax, errorbar=("se", 1))
    ax.set_ylabel(f'Fraction of top {100*kk:0.2f}%')
    plot_utils.set_size(w=1.2, h=1, ax=ax)
    fig.savefig(fig_dir / f'fraction_true_top_{kk}.pdf', bbox_inches='tight', transparent=True)


/tmp/ipykernel_1273208/3554573390.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(1,1)


## Antibiotics

In [42]:
storage = []
for file in result_dir.glob('antibiotics*batch20*.json'): 
    with open(file, 'r') as f: 
        storage.extend(json.load(f))

In [43]:
all_data = pd.read_csv(data_dir / 'antibiotics.csv')
all_smiles = list(all_data['smiles'])
test_data = {smi: score for smi, score in zip(all_smiles, all_data['Mean'])}

k = [0.01, 0.05, 0.1]
kN = [int(np.ceil(kk*len(test_data))) for kk in k]
top_smis = {kk: set(sorted(list(test_data), key = lambda smi: test_data[smi])[:kk]) for kk in kN}

for entry in storage: 
    for kkN, kk in zip(kN, k): 
        entry[f'Fraction top {100*kk:0.2f}%'] = len(top_smis[kkN].intersection(set(entry["All acquired points"].keys())))/kkN
    entry.pop("All acquired points")
    entry.pop("New acquired points")

data = pd.DataFrame(storage).sort_values(by='Method', key=lambda x: x.map(method_order))

In [44]:
fig_dir = Path('antibiotics')
fig_dir.mkdir(exist_ok=True, parents=True)

In [45]:
for kk in [1, 10, 50, 100]:
    fig, ax = plt.subplots(1,1)
    sns.lineplot(
        data=data, x='Iteration', y=f'Top {kk} ave', 
        hue='Method', style='Method', 
        ax=ax, palette=plot_utils.method_colors, errorbar=('se',1))
    ax.set_ylabel(f'Average of top {kk} acquired')
    plot_utils.set_size(w=1.2, h=1, ax=ax)
    fig.savefig(fig_dir / f'top_{kk}_average.pdf', bbox_inches='tight', transparent=True)

for kk, kkN in zip(k, kN):
    fig, ax = plt.subplots(1,1)
    sns.lineplot(
        data=data, x='Iteration', y=f'Fraction top {100*kk:0.2f}%', 
        hue='Method', style='Method', 
        ax=ax, palette=plot_utils.method_colors, errorbar=("se", 1))
    ax.set_ylabel(f'Fraction of top {100*kk:0.2f}%')
    plot_utils.set_size(w=1.2, h=1, ax=ax)
    fig.savefig(fig_dir / f'fraction_true_top_{kk}.pdf', bbox_inches='tight', transparent=True)